# Proyeco Final - Ciencia De Datos En Python - Universidad Galileo
##### Francisco Duarte - 17001004

## Scope
El proyecto es sobre las importaciones de vehículos a Guatemala durante el 2022, a la fecha de realziación de este proyecto solamente se cuenta con los datos para los meses de enero y febrero. Estos datos fueron obtenidos desde el portal de la Superintendencia de Administración Tributaria (SAT), son dos archivos extensión .txt, los datos se encuentra sperados por el símbolo "|"; pero se convertieron a csv para una mejor compatibilidad con workbench. Se realizó otro archivo que contiene las aduanas, estas fueron copiadas desde una tabla del portal de la SAT.

Para objetivos del proyectos uno de los archivos se importará a una base de datos en el servicio RDS de AWS, y los otros dos estarán alojadas en un contedor S3 de AWS. Se utilizará librerías como pandas para el proceso de de ETL, se construirá un datawarehouse para almacenar los datos ya procesados y se alojarán en un Redshift de AWS. RDS tendrá los datos para enero 2022 y S3 febrero 2022 y las aduanas

## Exploración

##### import necesarios

In [1]:
%load_ext sql

In [2]:
import pandas as pd
import os
import configparser
import sys
import boto3
from sqlalchemy import create_engine

### Conexiones

In [3]:
#Cargando el archivo de configuracion
config = configparser.ConfigParser()
config.read_file(open("config.cfg"))

In [4]:
#Conexión a la base de datos en RDS
mysql_connection = "mysql+pymysql://{}:{}@{}/{}".format(
config.get("RDS", "DB_USER"),
config.get("RDS", "DB_PASSWORD"),
config.get("RDS", "DB_HOST"),
config.get("RDS", "DB_NAME"))

In [5]:
%sql $mysql_connection

'Connected: admin@importacion-vehiculos'

In [47]:
#Conexión al datawarehouse en Redshift
redshift_connection = "postgresql://{}:{}@{}:{}/{}".format(
config.get("Redshift", "DB_USER"),
config.get("Redshift", "DB_PASSWORD"),
config.get("Redshift", "DB_HOST"),
config.get("Redshift", "DB_PORT"),
config.get("Redshift", "DB_NAME"))

In [48]:
%sql $redshift_connection

'Connected: awsuser@dev'

In [6]:
#Conexión a S3
s3_connection = boto3.resource(
service_name = "s3",
region_name = config.get("S3", "REGION"),
aws_access_key_id = config.get("S3", "ACCESS"),
aws_secret_access_key = config.get("S3", "SECRET"))

### Obteniendo los datos desde RDS y S3

#### RDS

In [7]:
rds_importacion_select = "SELECT * FROM importacion;"
dfImportacionRDS = pd.read_sql(rds_importacion_select, mysql_connection)
dfImportacionRDS.head()

,Pais de Proveniencia,Aduana de Ingreso,Fecha de la Poliza,Partida Arancelaria,Modelo del Vehiculo,Marca,Linea,Centimetros Cubicos,Distintivo,Tipo de Vehiculo,Tipo de Importador,Tipo Combustible,Asientos,Puertas,Tonelaje,Valor CIF,Impuesto
0,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0,767777.85,92133.34
1,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0,767777.85,92133.34
2,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0,767777.85,92133.34
3,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0,767777.85,92133.34
4,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0,767777.85,92133.34


#### S3

In [9]:
dfImportacionS3 = pd.DataFrame()
dfAduanasS3 = pd.DataFrame()

for remoteFile in s3_connection.Bucket(config.get("S3", "BUCKET_NAME")).objects.all():
    file = s3_connection.Bucket(config.get("S3", "BUCKET_NAME")).Object(remoteFile.key).get()
    if("importacion" in remoteFile.key):
       data = pd.read_csv(file["Body"], delimiter = "|")
       dfImportacionS3 = dfImportacionS3.append(data)
    else:
       data = pd.read_csv(file["Body"], delimiter = ";")
       dfAduanasS3 = dfAduanasS3.append(data)

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/974039967.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAduanasS3 = dfAduanasS3.append(data)
/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/974039967.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfImportacionS3 = dfImportacionS3.append(data)


In [11]:
dfImportacionS3.head()

,Pais de Proveniencia,Aduana de Ingreso,Fecha de la Poliza,Partida Arancelaria,Modelo del Vehiculo,Marca,Linea,Centimetros Cubicos,Distintivo,Tipo de Vehiculo,Tipo de Importador,Tipo Combustible,Asientos,Puertas,Tonelaje,Valor CIF,Impuesto,Unnamed: 17
0,INDIA,PUERTO QUETZAL,08/02/2022,8711209000,2023,TVS,APACHE RTR 160,159.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,1389415.13,166729.82,NaN
1,INDIA,PUERTO QUETZAL,08/02/2022,8711209000,2023,TVS,APACHE RTR 160,159.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,1389415.13,166729.82,NaN
2,INDIA,PUERTO QUETZAL,08/02/2022,8711209000,2023,TVS,APACHE RTR 160,159.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,1389415.13,166729.82,NaN
3,INDIA,PUERTO QUETZAL,08/02/2022,8711209000,2023,TVS,APACHE RTR 160,159.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,1389415.13,166729.82,NaN
4,INDIA,PUERTO QUETZAL,08/02/2022,8711209000,2023,TVS,APACHE RTR 160,159.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,1389415.13,166729.82,NaN


In [12]:
dfAduanasS3.head()

,Aduanas,Es Almacenadora
0,Aduana Tecún Umán 1,NaN
1,Aduana Integrada Agua Caliente,NaN
2,Aduana Integrada Corinto,NaN
3,Aduana Integrada El Florido,NaN
4,Aduana Puerto Quetzal,NaN


#### Explorando los datos obtenidos

##### Conteo de los dataframe

In [13]:
dfImportacionRDS.count()

Pais de Proveniencia    38988
Aduana de Ingreso       38988
Fecha de la Poliza      38988
Partida Arancelaria     38988
Modelo del Vehiculo     38988
Marca                   38988
Linea                   38988
Centimetros Cubicos     38983
Distintivo              37565
Tipo de Vehiculo        38988
Tipo de Importador      38988
Tipo Combustible        38988
Asientos                38988
Puertas                 38988
Tonelaje                38988
Valor CIF               38988
Impuesto                38988
dtype: int64

In [14]:
dfImportacionS3.count()

Pais de Proveniencia    37462
Aduana de Ingreso       37462
 Fecha de la Poliza     37462
Partida Arancelaria     37462
Modelo del Vehiculo     37462
Marca                   37462
Linea                   37462
Centimetros Cubicos     37453
Distintivo              37308
Tipo de Vehiculo        37462
Tipo de Importador      37462
Tipo Combustible        37462
Asientos                37462
Puertas                 37462
Tonelaje                37462
Valor CIF               37462
Impuesto                37462
Unnamed: 17                 0
dtype: int64

In [15]:
dfAduanasS3.count()

Aduanas            31
Es Almacenadora     8
dtype: int64

##### Valors distintos columna País de proviniencia

In [16]:
dfImportacionRDS["Pais de Proveniencia"].unique()

array(['CHINA', 'INDIA', 'CANADA', 'ARGENTINA', 'COREA DEL SUR',
       'ESTADOS UNIDOS', 'JAPON', 'REINO UNIDO', 'MEXICO', 'TAILANDIA',
       'ITALIA', 'BRASIL', 'ALEMANIA REP. FED.', 'INDONESIA', 'SUECIA',
       'FRANCIA', 'BELICE', 'TAIWAN', 'HUNGRIA', 'BELGICA', 'AUSTRIA',
       'SUDAFRICA', 'COMOROS', 'TURQUIA', 'SLOVAKIA', 'AUSTRALIA',
       'IRLANDA', 'ESPANA', 'COLOMBIA'], dtype=object)

In [17]:
dfImportacionS3["Pais de Proveniencia"].unique()

array(['INDIA', 'ESTADOS UNIDOS', 'CANADA', 'JAPON', 'CHINA',
       'REINO UNIDO', 'FRANCIA', 'ALEMANIA REP. FED.', 'ITALIA', 'MEXICO',
       'BELICE', 'COREA DEL SUR', 'BRASIL', 'INDONESIA', 'TAILANDIA',
       'U.R.S.S.', 'SUECIA', 'TAIWAN', 'AUSTRIA', 'AUSTRALIA', 'SUIZA',
       'ESPANA', 'KENYA', 'TURQUIA', 'SLOVAKIA', 'BELGICA', 'ARGENTINA',
       'HUNGRIA', 'PANAMA'], dtype=object)

##### Valores distintos Aduana ingreso

In [18]:
dfImportacionRDS["Aduana de Ingreso"].unique()

array(['PUERTO QUETZAL', 'EL CARMEN', 'G8, CENTRALSA', 'G1, INTEGRADA',
       'SANTO TOMAS DE CASTILLA', 'PUERTO BARRIOS', 'TECUN UMAN',
       'G4, ALSERSA', 'MELCHOR DE MENCOS', 'EL CEIBO', 'LA MESILLA',
       'VALLE NUEVO', 'PEDRO DE ALVARADO', 'G5, CEALSA', 'VEHICULOS',
       'ADUANA INTEGRADA AGUA CALIENTE', 'G7, ALCORSA', 'G3, ALPASA',
       'SAN CRISTOBAL', 'ADUANA INTEGRADA CORINTO', 'LA ERMITA'],
      dtype=object)

In [19]:
dfImportacionS3["Aduana de Ingreso"].unique()

array(['PUERTO QUETZAL', 'TECUN UMAN', 'G1, INTEGRADA', 'G8, CENTRALSA',
       'EL CARMEN', 'PUERTO BARRIOS', 'SANTO TOMAS DE CASTILLA',
       'MELCHOR DE MENCOS', 'VEHICULOS', 'LA MESILLA',
       'PEDRO DE ALVARADO', 'SAN CRISTOBAL', 'EL CEIBO',
       'ADUANA INTEGRADA AGUA CALIENTE', 'VALLE NUEVO', 'G3, ALPASA',
       'G7, ALCORSA', 'G4, ALSERSA', 'ADUANA INTEGRADA CORINTO'],
      dtype=object)

In [20]:
dfAduanasS3["Aduanas"].unique()

array(['Aduana Tecún Umán 1', 'Aduana Integrada Agua Caliente',
       'Aduana Integrada Corinto', 'Aduana Integrada El Florido',
       'Aduana Puerto Quetzal', 'Aduana Santo Tomas De Castilla',
       'Aduana Express Aereo', 'Aduana Central', 'Aduana Tecun Uman',
       'Aduana Puerto Barrios', 'Aduana Pedro De Alvarado',
       'Aduana San Cristobal',
       'Aduana Almacenadora Integrada (almacenadora)',
       'Aduana Agua Caliente', 'Aduana El Florido',
       'Aduana Alpasa (almacenadora)', 'Aduana Centralsa (almacenadora)',
       'Aduana La Ermita', 'Aduana El Carmen', 'Aduana Valle Nuevo',
       'Aduana Alcorsa (almacenadora)', 'Aduana Alminter (almacenadora)',
       'Aduana Alsersa (almacenadora)', 'Aduana Cealsa (almacenadora)',
       'Aduana Almaguate (almacenadora)', 'Aduana Central De Aviacion',
       'Aduana Melchor De Mencos', 'Aduana Fardos Postales', 'El Ceibo',
       'Aduana La Mesilla', 'Tikal'], dtype=object)

##### Valores distintos en la columna Marca

In [21]:
dfImportacionRDS["Marca"].unique()

array(['YAMAHA', 'HONDA', 'TOYOTA', 'KIA', 'CHEVROLET', 'HYUNDAI', 'JEEP',
       'SUZUKI', 'FORD', 'NISSAN', 'MOVESA', 'MAZDA', 'KAWASAKI',
       'MERCEDES-BENZ', 'FREIGHTLINER', 'SUBARU', 'ITALIKA', 'BAJAJ',
       'SATURN', 'MITSUBISHI', 'ISUZU', 'VOLKSWAGEN', 'MINI',
       'LAND ROVER', 'SCION', 'XCMG', 'BMW', 'AUDI', 'MITSUBISHI FUSO',
       'GMC', 'INTERNATIONAL', 'POLARIS', 'DODGE', 'SERPENTO', 'OSBORNE',
       'VOLVO', 'JOHN DEERE', 'ACURA', 'JAC', 'HARLEY-DAVIDSON', 'DUCATI',
       'ASIA HERO', 'UD NISSAN', 'PETERBILT', 'TRAILMOBILE', 'TVS',
       'LIUGONG', 'CHANGAN', 'FIAT', 'STRICK', 'HONDA FIT', 'UTILITY',
       'HOBBS', 'AZTEC', 'CADILLAC', 'CASE', 'HUMMER', 'INFINITI',
       'NEW HOLLAND', 'DORSEY', 'KENWORTH', 'PINES', 'KTM', 'PONTIAC',
       'WABASH', 'GEO', 'MCI', 'LEXUS', 'JCB', 'WRANGLER', 'SCOOTER',
       'PORSCHE', 'FREEDOM', 'CHRYSLER', 'CFMOTO', 'LOADCRAFT', 'GINDY',
       'HERCULES', 'BLUE BIRD', 'MASSEY FERGUSON', 'ZHONGYU', 'BOYD',
       'NEW FLYE

In [22]:
dfImportacionS3["Marca"].unique()

array(['TVS', 'HONDA', 'TOYOTA', 'PEUGEOT', 'SUZUKI', 'MAZDA', 'FORD',
       'FREIGHTLINER', 'BAJAJ', 'NISSAN', 'ISUZU', 'MITSUBISHI', 'HINO',
       'PETERBILT', 'UTILITY', 'PACER', 'INTERNATIONAL', 'JEEP',
       'VOLKSWAGEN', 'BMW', 'CHEVROLET', 'KIA', 'SCION', 'MINI', 'MOVESA',
       'JAC', 'HYUNDAI', 'STRICK', 'SERPENTO', 'STERLING', 'KENWORTH',
       'YAMAHA', 'HUMMER', 'KAWASAKI', 'FREEDOM', 'MACK', 'SUBARU',
       'MERCEDES-BENZ', 'PORSCHE', 'SINOTRUK', 'EDASAINFRA', 'RAM',
       'MONON', 'LAND ROVER', 'LEXUS', 'AUDI', 'HERO', 'ITALIKA', 'CASE',
       'ETNYRE', 'CONTI', 'HARLEY-DAVIDSON', 'SSANG YONG', 'LUFKIN',
       'UAZ', 'DODGE', 'GMC', 'ROAD SYSTEMS', 'VOLVO', 'FRUEHAUF',
       'POLARIS', 'DORSEY', 'UD NISSAN', 'MITSUBISHI FUSO', 'MCI', 'EAST',
       'BUICK', 'WABASH', 'CATERPILLAR', 'KTM', 'HUSQVARNA', 'CHRYSLER',
       'GAS-GAS', 'DUCATI', 'MG', 'BIRMINGHAM', 'TRAILMOBILE', 'GENESIS',
       'HAOJUE', 'KYMCO', 'JOHN DEERE', 'GREAT DANE', 'COTTRELL',
       'DON

##### Valores distintos en la columna Linea

In [23]:
dfImportacionRDS["Linea"].unique()

array(['T110C', 'NAVI 110', 'MATRIX', ..., 'TACOMA TRD SPORT D/C 4WD',
       'TRX420TM1 RANCHER 4X2', 'XL7 4WD'], dtype=object)

In [24]:
dfImportacionS3["Linea"].unique()

array(['APACHE RTR 160', 'CR-V EX 4WD', 'COROLLA', ..., 'TRX250TM RECON',
       'TACOMA DO/CAB SR5 4WD', 'CR-V REAL TIME 4WD'], dtype=object)

##### Valores distintos columna Tipo importador

In [25]:
dfImportacionRDS["Tipo de Importador"].unique()

array(['DISTRIBUIDOR', 'OCASIONAL'], dtype=object)

In [26]:
dfImportacionS3["Tipo de Importador"].unique()

array(['DISTRIBUIDOR', 'OCASIONAL'], dtype=object)

##### Relación entre columna Linea Marca

In [27]:
dfImportacionRDS.groupby(["Linea"])["Marca"].count()

Linea
1 TON 2WD             1
1 TON LONG BED 2WD    2
1085 4X2              1
125 RR-S CHALEE       1
128i COUPE            1
                     ..
xA                    3
xA HATCHBACK          1
xB                    8
xD                    5
xD HATCHBACK          1
Name: Marca, Length: 2813, dtype: int64

##### Columna Distintivo

In [28]:
dfImportacionRDS["Distintivo"].unique()

array(['LIVIANO', 'PESADO', None], dtype=object)

In [29]:
dfImportacionS3["Distintivo"].unique()

array(['LIVIANO', 'PESADO', nan], dtype=object)

#### Análisis exploración

Como resultado de la exploración de los datos se pudo encontrar que para la imortación hecha desde S3, hay una columna que se puede descartar ya que no posee ningún valor. También se determinó que diferentes marcas pueden tener las mismas líneas, el distintivo en algunas ocasiones viene con un null, así como en la aduana de ingreso en ocasiones contiene valores incorrectos, por ejemplo: "vehiculos", en relación a las aduanas los datos obtenidos del archivo Aduandas alojado en S3, el nombre de la aduanda es del tipo "Aduana Puerto Barrarios" mientras que en RDS y el otro archivo en S3 tiene las aduanas como "Puerto Barrios", otra diferencia es columna "Es almacenadora" ya que las importaciones de vehículos no trae esta información y es una columna que para este caso no aporta mayor información por lo que puede ser eliminada.


## Modelo De Datos

Como resultado del análisis anterior se creará el siguiente modelo:
- dimensión Paises
- dimensión Aduanas
- dimensión Partida Arancelaria
- dimensión Fecha
- dimensión Tipo Importador
- dimensión Vehiculo
- tabla de hechos Importaciones

## Procesamiento

##### Uniendo los dataframe de importaciones

In [10]:
dfImportaciones = pd.DataFrame()

dfImportaciones = dfImportaciones.append(dfImportacionRDS)

#Cambio de nombre columna Fecha de la Poliza en dfImportacionS3 para match con dfImportacionRDS
dfImportacionS3 = dfImportacionS3.rename(columns = {" Fecha de la Poliza" : "Fecha de la Poliza" })
dfImportaciones = dfImportaciones.append(dfImportacionS3.iloc[:,0:17])
dfImportaciones.count()

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/3100517111.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfImportaciones = dfImportaciones.append(dfImportacionRDS)
/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/3100517111.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfImportaciones = dfImportaciones.append(dfImportacionS3.iloc[:,0:17])


Pais de Proveniencia    76450
Aduana de Ingreso       76450
Fecha de la Poliza      76450
Partida Arancelaria     76450
Modelo del Vehiculo     76450
Marca                   76450
Linea                   76450
Centimetros Cubicos     76436
Distintivo              74873
Tipo de Vehiculo        76450
Tipo de Importador      76450
Tipo Combustible        76450
Asientos                76450
Puertas                 76450
Tonelaje                76450
Valor CIF               76450
Impuesto                76450
dtype: int64

In [31]:
dfImportaciones.head()

,Pais de Proveniencia,Aduana de Ingreso,Fecha de la Poliza,Partida Arancelaria,Modelo del Vehiculo,Marca,Linea,Centimetros Cubicos,Distintivo,Tipo de Vehiculo,Tipo de Importador,Tipo Combustible,Asientos,Puertas,Tonelaje,Valor CIF,Impuesto
0,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,767777.85,92133.34
1,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,767777.85,92133.34
2,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,767777.85,92133.34
3,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,767777.85,92133.34
4,CHINA,PUERTO QUETZAL,18/01/2022,8711209000,2022,YAMAHA,T110C,110.0,LIVIANO,MOTO,DISTRIBUIDOR,GASOLINA,2,0,0.0,767777.85,92133.34


##### Creando dataframe para las aduanas

In [11]:
dfAduanas = pd.DataFrame()

dfAduanas = pd.concat([dfAduanas, dfAduanasS3["Aduanas"].str.upper()], axis = 1)
dfAduanas = dfAduanas.rename(columns = {"Aduanas": "Aduana"})
dfAduanas["Nombre Corto"] = [aduana.split("ADUANA")[1].strip() if len(aduana.split("ADUANA")) == 2 else aduana for aduana in dfAduanas["Aduana"]]
dfAduanas.head()

,Aduana,Nombre Corto
0,ADUANA TECÚN UMÁN 1,TECÚN UMÁN 1
1,ADUANA INTEGRADA AGUA CALIENTE,INTEGRADA AGUA CALIENTE
2,ADUANA INTEGRADA CORINTO,INTEGRADA CORINTO
3,ADUANA INTEGRADA EL FLORIDO,INTEGRADA EL FLORIDO
4,ADUANA PUERTO QUETZAL,PUERTO QUETZAL


In [12]:
dfTempAduanas = pd.DataFrame({"Aduanas" :dfImportaciones["Aduana de Ingreso"].unique()})

for item in dfTempAduanas["Aduanas"]:
    if(item  not in dfAduanas["Nombre Corto"].values and item not in dfAduanas["Aduana"].values):
        dfAduanas =dfAduanas.append({"Aduana":item, "Nombre Corto":item}, ignore_index = True)
        
dfAduanas

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/1776890133.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAduanas =dfAduanas.append({"Aduana":item, "Nombre Corto":item}, ignore_index = True)
/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/1776890133.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAduanas =dfAduanas.append({"Aduana":item, "Nombre Corto":item}, ignore_index = True)
/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/1776890133.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfAduanas =dfAduanas.append({"Aduana":item, "Nombre Corto":item}, ignore_index = True)
/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/1776890133.py:5: FutureWarning:

,Aduana,Nombre Corto
0,ADUANA TECÚN UMÁN 1,TECÚN UMÁN 1
1,ADUANA INTEGRADA AGUA CALIENTE,INTEGRADA AGUA CALIENTE
2,ADUANA INTEGRADA CORINTO,INTEGRADA CORINTO
3,ADUANA INTEGRADA EL FLORIDO,INTEGRADA EL FLORIDO
4,ADUANA PUERTO QUETZAL,PUERTO QUETZAL
5,ADUANA SANTO TOMAS DE CASTILLA,SANTO TOMAS DE CASTILLA
6,ADUANA EXPRESS AEREO,EXPRESS AEREO
7,ADUANA CENTRAL,CENTRAL
8,ADUANA TECUN UMAN,TECUN UMAN
9,ADUANA PUERTO BARRIOS,PUERTO BARRIOS


### Creando las Dimensiones

##### Dimensión Aduanas

In [13]:
dimAduana = pd.DataFrame(columns= ["aduana_sk"])

dimAduana = dimAduana.append(dfAduanas)
dimAduana = dimAduana.rename(columns = {"Aduana":"aduana", "Nombre Corto":"nombre_corto"})
dimAduana["aduana_sk"] = range(1, len(dimAduana) + 1)
dimAduana.head()

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/3749969793.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dimAduana = dimAduana.append(dfAduanas)


,aduana_sk,aduana,nombre_corto
0,1,ADUANA TECÚN UMÁN 1,TECÚN UMÁN 1
1,2,ADUANA INTEGRADA AGUA CALIENTE,INTEGRADA AGUA CALIENTE
2,3,ADUANA INTEGRADA CORINTO,INTEGRADA CORINTO
3,4,ADUANA INTEGRADA EL FLORIDO,INTEGRADA EL FLORIDO
4,5,ADUANA PUERTO QUETZAL,PUERTO QUETZAL


##### Dimensión Fecha

In [14]:
dimFecha = pd.DataFrame()

dimFecha = pd.concat([dimFecha, dfImportaciones["Fecha de la Poliza"]], axis = 1)
dimFecha = dimFecha.rename(columns = {"Fecha de la Poliza": "fecha"})
dimFecha = dimFecha.drop_duplicates(subset = "fecha")
dimFecha["fecha"] = pd.to_datetime(dimFecha["fecha"], format = "%d/%m/%Y")
dimFecha.head()

,fecha
0,2022-01-18
9,2022-01-19
44,2022-01-26
54,2022-01-27
210,2022-02-07


In [15]:
dimFecha["anio"] = pd.DatetimeIndex(dimFecha["fecha"]).year
dimFecha["mes"] = pd.DatetimeIndex(dimFecha["fecha"]).month
dimFecha["trimestre"] = pd.DatetimeIndex(dimFecha["fecha"]).quarter
dimFecha["dia"] = pd.DatetimeIndex(dimFecha["fecha"]).day
dimFecha["semana"] = pd.DatetimeIndex(dimFecha["fecha"]).week
dimFecha["dia_semana"] = pd.DatetimeIndex(dimFecha["fecha"]).dayofweek
dimFecha.head()

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/2673028398.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFecha["semana"] = pd.DatetimeIndex(dimFecha["fecha"]).week


,fecha,anio,mes,trimestre,dia,semana,dia_semana
0,2022-01-18,2022,1,1,18,3,1
9,2022-01-19,2022,1,1,19,3,2
44,2022-01-26,2022,1,1,26,4,2
54,2022-01-27,2022,1,1,27,4,3
210,2022-02-07,2022,2,1,7,6,0


##### Dimensión Tipo Importador

In [16]:
dimTipoImportador = pd.DataFrame(columns = ["tipo_importador_sk"])

dimTipoImportador = pd.concat([dimTipoImportador, dfImportaciones["Tipo de Importador"].drop_duplicates()])
dimTipoImportador = dimTipoImportador.rename(columns = {0:"tipo_importador"})
dimTipoImportador["tipo_importador_sk"] = range(1, len(dimTipoImportador)+1)
dimTipoImportador.head()

,tipo_importador_sk,tipo_importador
0,1,DISTRIBUIDOR
9,2,OCASIONAL


##### Dimensión Partida arancelaria

In [17]:
dimPartidaArancelaria = pd.DataFrame(columns = ["partida_arancelaria_sk"])

dimPartidaArancelaria = pd.concat([dimPartidaArancelaria, dfImportaciones["Partida Arancelaria"]])
dimPartidaArancelaria = dimPartidaArancelaria.rename(columns = {0: "partida_arancelaria"})
dimPartidaArancelaria["partida_arancelaria"] = pd.to_numeric(dimPartidaArancelaria["partida_arancelaria"], downcast = "integer")
dimPartidaArancelaria = dimPartidaArancelaria.drop_duplicates()
dimPartidaArancelaria["partida_arancelaria_sk"] = range(1, len(dimPartidaArancelaria)+1)
dimPartidaArancelaria.head()

,partida_arancelaria_sk,partida_arancelaria
0,1,8711209000
44,2,8703236991
45,3,8704215900
52,4,8703326991
53,5,8703237991


##### Dimensión Países

In [18]:
dimPais = pd.DataFrame(columns = ["pais_sk"])

dimPais = pd.concat([dimPais, dfImportaciones["Pais de Proveniencia"].drop_duplicates()])
dimPais = dimPais.rename(columns = {0: "pais"})
dimPais["pais_sk"] = range(1, len(dimPais) + 1)
dimPais.head()

,pais_sk,pais
0,1,CHINA
9,2,INDIA
44,3,CANADA
45,4,ARGENTINA
52,5,COREA DEL SUR


##### Dimensión Vehiculos

In [19]:
dimVehiculo = pd.DataFrame(columns = ["vehiculo_sk"])

dimVehiculo = pd.concat([dimVehiculo, dfImportaciones.loc[:,["Modelo del Vehiculo", "Marca", "Linea", "Centimetros Cubicos", "Distintivo", 
                                                      "Tipo de Vehiculo", "Tipo Combustible", "Asientos", "Puertas", "Tonelaje"]]])
dimVehiculo = dimVehiculo.rename(columns = {"Modelo del Vehiculo": "modelo", "Marca":"marca", "Linea":"linea", "Centimetros Cubicos" : "centimetros_cubicos",
                                         "Distintivo":"distintivo", "Tipo de Vehiculo": "tipo", "Tipo Combustible": "combustible", "Asientos": "asientos",
                                         "Puertas": "puertas", "Tonelaje": "tonelaje"})
dimVehiculo = dimVehiculo.drop_duplicates()
dimVehiculo["vehiculo_sk"] = range(1, len(dimVehiculo) +1)
dimVehiculo.head()

,vehiculo_sk,modelo,marca,linea,centimetros_cubicos,distintivo,tipo,combustible,asientos,puertas,tonelaje
0,1,2022.0,YAMAHA,T110C,110.0,LIVIANO,MOTO,GASOLINA,2.0,0.0,0.0
9,2,2021.0,HONDA,NAVI 110,109.0,LIVIANO,MOTO,GASOLINA,2.0,0.0,0.0
44,3,2005.0,TOYOTA,MATRIX,1800.0,LIVIANO,CAMIONETILLA,GASOLINA,5.0,4.0,0.0
45,4,2022.0,TOYOTA,HILUX,2393.0,LIVIANO,PICK UP,DIESEL,3.0,2.0,1.0
52,5,2007.0,KIA,SPORTAGE TLX 2WD,2000.0,LIVIANO,CAMIONETA,DIESEL,5.0,5.0,0.0


### Creando la Tabla de Hechos

In [20]:
factImportaciones = pd.DataFrame()

factImportaciones = factImportaciones.append(dfImportaciones)
factImportaciones["Fecha de la Poliza"] = pd.to_datetime(factImportaciones["Fecha de la Poliza"], format = "%d/%m/%Y")
factImportaciones = factImportaciones.rename(columns = {"Valor CIF": "valor_cif", "Impuesto":"impuesto"})

#merge fact y dimensiones
factImportaciones = factImportaciones.merge(dimPais, left_on="Pais de Proveniencia", right_on="pais").drop(columns = ["Pais de Proveniencia", "pais"])
factImportaciones = factImportaciones.merge(dimAduana[["aduana_sk", "nombre_corto"]], left_on="Aduana de Ingreso", right_on="nombre_corto").drop(columns = ["Aduana de Ingreso", "nombre_corto"])
factImportaciones = factImportaciones.merge(dimFecha[["fecha"]], left_on="Fecha de la Poliza", right_on="fecha").drop(columns = ["Fecha de la Poliza"])
factImportaciones = factImportaciones.merge(dimTipoImportador, left_on="Tipo de Importador", right_on="tipo_importador").drop(columns = ["Tipo de Importador", "tipo_importador"])
factImportaciones = factImportaciones.merge(dimVehiculo, left_on=["Modelo del Vehiculo", "Marca", "Linea", "Centimetros Cubicos", "Distintivo", 
                                                      "Tipo de Vehiculo", "Tipo Combustible", "Asientos", "Puertas", "Tonelaje"], right_on=["modelo", "marca", "linea", "centimetros_cubicos", "distintivo", 
                                                      "tipo", "combustible", "asientos", "puertas", "tonelaje"]).drop(columns = ["Modelo del Vehiculo", "Marca", "Linea", "Centimetros Cubicos", "Distintivo", 
                                                      "Tipo de Vehiculo", "Tipo Combustible", "Asientos", "Puertas", "Tonelaje","modelo", "marca", "linea", "centimetros_cubicos", "distintivo", 
                                                      "tipo", "combustible", "asientos", "puertas", "tonelaje"])
factImportaciones = factImportaciones.merge(dimPartidaArancelaria, left_on="Partida Arancelaria", right_on="partida_arancelaria").drop(columns = ["Partida Arancelaria", "partida_arancelaria"])

factImportaciones = factImportaciones[["fecha", "pais_sk", "aduana_sk", "partida_arancelaria_sk", "tipo_importador_sk", "vehiculo_sk","valor_cif", "impuesto"]]
factImportaciones

/var/folders/hh/25c_zdz92t9c70mlh69s5tg80000gn/T/ipykernel_887/614960734.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  factImportaciones = factImportaciones.append(dfImportaciones)


,fecha,pais_sk,aduana_sk,partida_arancelaria_sk,tipo_importador_sk,vehiculo_sk,valor_cif,impuesto
0,2022-01-18,1,5,1,1,1,767777.85,92133.34
1,2022-01-18,1,5,1,1,1,767777.85,92133.34
2,2022-01-18,1,5,1,1,1,767777.85,92133.34
3,2022-01-18,1,5,1,1,1,767777.85,92133.34
4,2022-01-18,1,5,1,1,1,767777.85,92133.34
...,...,...,...,...,...,...,...,...
76403,2022-03-07,7,5,113,2,9251,324070.82,38888.50
76404,2022-03-07,7,5,113,2,9251,324070.67,38888.48
76405,2022-03-07,7,5,113,2,9251,324070.67,38888.48
76406,2022-02-28,6,6,112,2,8877,404665.71,48559.89


### Cargando a Redshift

In [52]:
connection_engine = create_engine(redshift_connection)

In [55]:
#Insert de las dimensiones y tabla de hechos
dimAduana.to_sql("dim_aduana", connection_engine, index = False, if_exists = "append")
dimFecha.to_sql("dim_fecha", connection_engine, index = False, if_exists = "append", method="multi")
dimTipoImportador.to_sql("dim_tipo_importador", connection_engine, index = False, if_exists = "append")
dimPartidaArancelaria.to_sql("dim_partida_arancelaria", connection_engine, index = False, if_exists = "append", method="multi")
dimPais.to_sql("dim_pais", connection_engine, index = False, if_exists = "append", method="multi")
dimVehiculo.to_sql("dim_vehiculo", connection_engine, index = False, if_exists = "append", method="multi")
factImportaciones.to_sql("fact_importaciones", connection_engine, index = False, if_exists = "append", method="multi")

76408

## Analitica

##### ¿Cuál fue el monto recaudado en impuesto para enero y febrero de 2022?

In [56]:
sum(factImportaciones["impuesto"])

8297207141.788323

El impuesto recaudado para enero y febrero 2022 es de Q. 8,297,207,141.79

##### ¿Cuánto es el impuesto recaudado por día?

In [152]:
factImportaciones.groupby("fecha")["impuesto"].sum()

fecha
2022-01-08    5.954568e+05
2022-01-09    5.410548e+04
2022-01-10    1.482876e+08
2022-01-11    2.084106e+08
2022-01-12    1.519079e+08
2022-01-13    9.263677e+07
2022-01-14    2.855507e+08
2022-01-15    7.303961e+05
2022-01-16    1.362530e+05
2022-01-17    7.940829e+07
2022-01-18    6.323541e+08
2022-01-19    2.571232e+08
2022-01-20    1.978454e+07
2022-01-21    1.640633e+08
2022-01-22    8.990819e+05
2022-01-23    2.743252e+05
2022-01-24    3.005043e+07
2022-01-25    4.441352e+08
2022-01-26    7.356601e+08
2022-01-27    4.271329e+07
2022-01-28    3.217900e+08
2022-01-29    1.701847e+07
2022-01-30    4.126262e+05
2022-01-31    5.403526e+07
2022-02-01    2.276849e+08
2022-02-02    2.933588e+08
2022-02-03    1.456622e+07
2022-02-04    1.414441e+07
2022-02-05    1.945745e+07
2022-02-06    5.021689e+05
2022-02-07    1.631682e+08
2022-02-08    3.188953e+08
2022-02-09    5.583870e+08
2022-02-10    3.248286e+08
2022-02-11    1.531459e+08
2022-02-12    1.659576e+07
2022-02-13    4.053793

##### ¿Cuál el valor promedio de la importaciones?

In [154]:
factImportaciones["valor_cif"].mean()

904913.0542316281

El valor CIF promedio de las importaciones es de Q. 904,913.05

##### ¿Cuál el monto de la importación de mayor valor?

In [155]:
factImportaciones["valor_cif"].max()

5199182.49

La importación de mayor valor es de Q. 5,199,182.49

##### ¿Cuál es el monto de la importación de menor valor?

In [21]:
factImportaciones["valor_cif"].min()

1964.84

La importación de menor valor es de Q. 1,964.84

##### ¿Cuánto es el país de donde más importa?

In [28]:
dfTemp = pd.DataFrame()
dfTemp = pd.concat([dfTemp,factImportaciones["pais_sk"]], axis =1)
dfTemp = dfTemp.merge(dimPais, left_on="pais_sk", right_on="pais_sk")
dfTemp.groupby("pais")["pais_sk"].count().sort_values(ascending= False)

pais
CHINA                 32465
INDIA                 20486
JAPON                  9856
ESTADOS UNIDOS         5272
COREA DEL SUR          2042
CANADA                 1491
MEXICO                 1127
TAILANDIA               873
BRASIL                  572
ARGENTINA               486
REINO UNIDO             457
ALEMANIA REP. FED.      360
INDONESIA               313
TAIWAN                  215
AUSTRIA                 117
FRANCIA                  74
ITALIA                   73
BELICE                   21
SUECIA                   20
HUNGRIA                  16
TURQUIA                  16
IRLANDA                  15
SLOVAKIA                 11
BELGICA                   8
AUSTRALIA                 5
ESPANA                    5
SUDAFRICA                 3
SUIZA                     3
PANAMA                    2
COMOROS                   1
KENYA                     1
COLOMBIA                  1
U.R.S.S.                  1
Name: pais_sk, dtype: int64

El país de donde más se importan vehículos es China

##### ¿Cuántos vehiculos ingresan por aduana?

In [33]:
dfTemp = pd.DataFrame()
dfTemp = pd.concat([dfTemp,factImportaciones["aduana_sk"]], axis =1)
dfTemp = dfTemp.merge(dimAduana, left_on="aduana_sk", right_on="aduana_sk")
dfTemp.groupby("aduana")["aduana_sk"].count().sort_values(ascending= False)

aduana
ADUANA PUERTO QUETZAL             57571
ADUANA EL CARMEN                   6352
ADUANA TECUN UMAN                  4770
ADUANA SANTO TOMAS DE CASTILLA     3192
ADUANA PUERTO BARRIOS              2478
G8, CENTRALSA                       902
G1, INTEGRADA                       711
ADUANA PEDRO DE ALVARADO            294
ADUANA MELCHOR DE MENCOS             36
G4, ALSERSA                          19
G7, ALCORSA                          19
G3, ALPASA                           18
VEHICULOS                            16
ADUANA LA MESILLA                     8
EL CEIBO                              6
G5, CEALSA                            6
ADUANA SAN CRISTOBAL                  5
ADUANA VALLE NUEVO                    4
ADUANA LA ERMITA                      1
Name: aduana_sk, dtype: int64

La aduana donde ingresan más vehículos es la Aduana de Puerto Quetzal